In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/DLProjects/JTM

Mounted at /content/gdrive
/content/gdrive/My Drive/DLProjects/JTM


In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a062ef60-ba2e-43ff-353b-c3e5efcfd094)


In [6]:
import torch
from dataloader import AudioBatchData
from model import CPCEncoder, CPCAR, CPCModel, CPCUnsupersivedCriterion
from trainer import run
from datetime import datetime
import os

In [4]:
labelsBy = 'ensemble'
print("Loading the training dataset")
trainDataset = AudioBatchData(rawAudioPath='data/musicnet_lousy/train_data',
                                metadataPath='data/musicnet_lousy/metadata_train.csv',
                                sizeWindow=20480,
                                labelsBy=labelsBy,
                                outputPath='data/musicnet_lousy/train_data/train',
                                CHUNK_SIZE=1e9,
                                NUM_CHUNKS_INMEM=7)
print("Training dataset loaded")
print("")

print("Loading the validation dataset")
valDataset = AudioBatchData(rawAudioPath='data/musicnet_lousy/train_data',
                            metadataPath='data/musicnet_lousy/metadata_val.csv',
                            sizeWindow=20480,
                            labelsBy=labelsBy,
                            outputPath='data/musicnet_lousy/train_data/val',
                            CHUNK_SIZE=1e9,
                            NUM_CHUNKS_INMEM=1)
print("Validation dataset loaded")
print("")

Loading the training dataset
Chunks already exist at data/musicnet_lousy/train_data/train/ensemble
Loading files
Loaded 288 sequences, elapsed=155.862 secs
Training dataset loaded

Loading the validation dataset
Chunks already exist at data/musicnet_lousy/train_data/val/ensemble
Loading files
Loaded 32 sequences, elapsed=18.488 secs
Validation dataset loaded



In [ ]:
samplingType = 'samesequence'

# Encoder network
encoderNet = CPCEncoder(512, 'layerNorm')
# AR Network
arNet = CPCAR(512, 256, samplingType == 'sequential', 1, mode="GRU", reverse=False)

cpcModel = CPCModel(encoderNet, arNet)
batchSize = 8
cpcModel.supervised = False

cpcCriterion = CPCUnsupersivedCriterion(nPredicts=12,
                                        dimOutputAR=256,
                                        dimOutputEncoder=512,
                                        negativeSamplingExt=128,
                                        mode=None,
                                        dropout=False)
cpcCriterion.cuda()
cpcModel.cuda()
gParams = list(cpcCriterion.parameters()) + list(cpcModel.parameters())
lr = 2e-4
optimizer = torch.optim.Adam(gParams, lr=lr, betas=(0.9, 0.999), eps=1e-8)

expDescription = f'{samplingType}_'
if samplingType == 'samecategory':
    expDescription += f'{labelsBy}_'

pathCheckpoint = f'logs/{expDescription}{datetime.now().strftime("%d-%m_%H-%M-%S")}'
# if not os.path.isdir(pathCheckpoint):
os.makedirs(pathCheckpoint, exist_ok=True)
pathCheckpoint = os.path.join(pathCheckpoint, "checkpoint")

logs = {"epoch": [], "iter": [], "saveStep": 1, "logging_step": 1000}
run(trainDataset, valDataset, batchSize, samplingType, cpcModel, cpcCriterion, 30, optimizer, pathCheckpoint, logs)

Running 30 epochs
Starting epoch 0
Training dataset 10681 batches, Validation dataset 1172 batches, batch size 8
Update 1000
elapsed: 277.2 s
277.2 ms per batch, 34.7 ms / example

--------------------------------------------------
Training loss
            Step                1                2                3                4                5                6                7                8                9               10               11               12
   locLoss_train         4.840637         4.840911         4.841719         4.842169         4.842353         4.842612         4.842544         4.843078         4.843147         4.843249         4.843182         4.833532
            Step                1                2                3                4                5                6                7                8                9               10               11               12
    locAcc_train         0.008723         0.008676         0.008721         0.008575       